In [1]:
# This notebook is heavily based on the methodology of the notebook here: https://github.com/evolutionaryscale/esm/blob/main/examples/gfp_design.ipynb
# The notebook is similar to what the authors' did to generate a new candidate GFP protein, so it seemed like a sensible methodology to start with


# Tweak these variables to change different model parameters
protein_pdb_id = "1qy3"  # the PDB id of the protein we want to run inference on
protein_chain_id = "A"  # the chain id of the protein we want to run inference on
percent_masked = 0.7  # ratio of masked amino acids, from [0.0, 1.0]
num_seqs = 10  # how many sequences to generate

model_name = "esm3-medium-2024-03"  # the name of the model, change this if you want a bigger/smaller model
# other options: esm3-large-2024-03, esm3-medium-2024-08, esm3-small-2024-03, etc. 
model_temperature = 1.0 # a value in [0.0, 1.0?], higher numbers have more stochasticity

In [2]:
from IPython.display import clear_output
!pip install git+https://github.com/evolutionaryscale/esm.git
!pip install py3Dmol
clear_output()  # Suppress pip install log lines after installation is complete.

In [3]:
import biotite.sequence as seq
import biotite.sequence.align as align
import biotite.sequence.graphics as graphics
from getpass import getpass
import matplotlib.pyplot as pl
import py3Dmol
import torch

from esm.sdk import client
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig
from esm.utils.structure.protein_chain import ProteinChain

In [5]:
# token: get from EvolutionaryScale Forge
token = getpass("Token from Forge console: ")

Token from Forge console:  ········


In [6]:
model = client(
    model=model_name,
    url="https://forge.evolutionaryscale.ai",
    token=token,
)

In [7]:
template_gfp = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb(protein_pdb_id, chain_id=protein_chain_id)
)

print("Original Sequence:")
print(template_gfp.sequence)

Original Sequence:
KGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQEATISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGI


In [12]:
%%time

import csv
import random 
from tqdm import tqdm

def mask_sequence(s: str, fraction: float) -> str: 
    if not (0.0 <= fraction <= 1.0):
        raise ValueError("Fraction must be between 0.0 and 1.0")
    num_to_mask = int(len(s) * fraction)
    indices_to_mask = random.sample(range(len(s)), num_to_mask)
    s_list = list(s)
    for i in indices_to_mask:
        s_list[i] = "_"
    return ''.join(s_list)


# Set up the CSV file for logging
csv_file = "generation_results.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Sequence_ID", "Generated_Sequence", "Sequence_Identity", "Backbone_RMSD"])

template_gfp_tokens = model.encode(template_gfp)

# Loop through the generation and logging process
for i in tqdm(range(num_seqs), desc="Generating Sequences"):
    prompt_sequence = mask_sequence(template_gfp.sequence, percent_masked)
    prompt = model.encode(
        ESMProtein(sequence=prompt_sequence)
    )
    # Copy over the same structure
    prompt.structure = template_gfp_tokens.structure
    
    # Generation of sequence
    num_tokens_to_decode = (prompt.sequence == 32).sum().item()
    sequence_generation = model.generate(
        prompt,
        GenerationConfig(
            track="sequence",
            num_steps=num_tokens_to_decode,
            temperature=model_temperature,
        )
    )

    # Generate a new structure from the sequence
    sequence_generation.structure = None
    length_of_sequence = sequence_generation.sequence.numel() - 2
    sequence_generation = model.generate(
        sequence_generation,
        GenerationConfig(
            track="structure",
            num_steps=length_of_sequence,
            temperature=0.0,
        )
    )

    # Decode to AA string and coordinates.
    sequence_generation_protein = model.decode(sequence_generation)
    generated_sequence = sequence_generation_protein.sequence
    print(f"Generated Sequence {i+1}: {generated_sequence}")

    # Sequence alignment and identity calculation
    seq1 = seq.ProteinSequence(template_gfp.sequence)
    seq2 = seq.ProteinSequence(generated_sequence)
    alignments = align.align_optimal(
        seq1,
        seq2,
        align.SubstitutionMatrix.std_protein_matrix(),
        gap_penalty=(-10, -1),
    )
    alignment = alignments[0]
    identity = align.get_sequence_identity(alignment)
    sequence_identity = 100 * identity
    print(f"Sequence {i+1} Identity: {sequence_identity:.2f}%")

    # RMSD calculation
    template_chain = template_gfp.to_protein_chain()
    generation_chain = sequence_generation_protein.to_protein_chain()
    backbone_rmsd = template_chain.rmsd(generation_chain)
    print(f"Backbone RMSD {i+1}: {backbone_rmsd:.2f}")

    # Log the results in the CSV file
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f"Sequence_{i+1}", generated_sequence, f"{sequence_identity:.2f}%", f"{backbone_rmsd:.2f}"])

print("Generation complete and results logged in generation_results.csv")


Generating Sequences:  10%|█         | 1/10 [01:24<12:40, 84.49s/it]

Generated Sequence 1: KGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTLGAQCFSRYPDHMKQHDFFKSAMPEGYVQERTISFKDDGNYKIRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGI
Sequence 1 Identity: 98.24%
Backbone RMSD 1: 0.65


Generating Sequences:  20%|██        | 2/10 [02:49<11:18, 84.77s/it]

Generated Sequence 2: KGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLXHGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGI
Sequence 2 Identity: 97.80%
Backbone RMSD 2: 0.65


Generating Sequences:  30%|███       | 3/10 [04:14<09:52, 84.69s/it]

...

Generated Sequence 8: KGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKGNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGI
Sequence 8 Identity: 98.24%
Backbone RMSD 8: 0.72


Generating Sequences:  90%|█████████ | 9/10 [12:42<01:24, 84.85s/it]

Generated Sequence 9: KGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLQYGIYCFARYPDHMKRHDFFKSAMPEGYVQERTISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYRQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGI
Sequence 9 Identity: 96.92%
Backbone RMSD 9: 0.73


Generating Sequences: 100%|██████████| 10/10 [14:07<00:00, 84.78s/it]

Generated Sequence 10: MGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLCYGVQCFSRYPDHMKQHDFFKSAMPEGYVQEATISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSLQSALSKDPNEKRDHMVLLEFVTAAGI
Sequence 10 Identity: 98.68%
Backbone RMSD 10: 0.71
Generation complete and results logged in generation_results.csv
CPU times: user 5.97 s, sys: 258 ms, total: 6.22 s
Wall time: 14min 9s
